In [1]:
import pandas as pd
import numpy as np
from geopy.distance import distance

In [2]:
df = pd.read_csv('./data/df_features.gz')

In [3]:
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng', 'Diff', 'Diff_Percentage',
       'Review_Month', 'Review_Year', 'Country', 'City', 'Pet', 'Purpose',
       'Whom', 'Room', 'Length', 'Device', 'Room_Recode', 'Nationality_Recode',
       'Length_Recode', 'Close_Landmarks', 'Dist_Center'],
      dtype='object')

### Airports

In [4]:
airports = pd.read_excel('./data/landmarks.xlsx', sheet_name='airports')
airports

,Latitude,Longitude,City
0,45.630063,8.725531,Milan
1,41.297449,2.081105,Barcelona
2,48.112616,16.573325,Vienna
3,49.014275,2.541627,Paris
4,52.310505,4.767814,Amsterdam
5,51.471129,-0.455318,London


In [5]:
df_compact = df[['Hotel_Address','lat','lng','City']].groupby(['Hotel_Address']).min().reset_index()

In [6]:
result = pd.merge(df_compact, airports, on='City')
result = result.dropna()

In [7]:
result['Dist_Airport'] = result.apply(lambda x: distance((x[1],x[2]), (x[4],x[5])).m, axis=1)
result = result[['Hotel_Address','Dist_Airport']]
result.head()

,Hotel_Address,Dist_Airport
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,11533.734110
1,2e Hugo de Grootstraat 2 6 Westerpark 1052 LC ...,10173.031663
2,Albert Cuypstraat 2 6 Oud Zuid 1072 CT Amsterd...,9512.220621
3,Aletta Jacobslaan 7 Slotervaart 1066 BP Amster...,5897.971066
4,Amstelstraat 17 Amsterdam City Center 1017 DA ...,10864.897108


In [8]:
df_updated = pd.merge(df, result, on='Hotel_Address', how='left')

In [9]:
df_updated.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng', 'Diff', 'Diff_Percentage',
       'Review_Month', 'Review_Year', 'Country', 'City', 'Pet', 'Purpose',
       'Whom', 'Room', 'Length', 'Device', 'Room_Recode', 'Nationality_Recode',
       'Length_Recode', 'Close_Landmarks', 'Dist_Center', 'Dist_Airport'],
      dtype='object')

#### Save Results

In [10]:
df_updated.to_csv("./data/df_features.gz", index_label=False, compression="gzip")